In [ ]:
!pip install catboost

import pandas as pd
import numpy as np
import joblib
from catboost import CatBoostClassifier, CatBoostRegressor, Pool

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 8.4 MB/s eta 0:00:00


In [ ]:
# 사용자로부터 입력 받기
def get_user_input():
    year = int(input("예측할 연도를 입력하세요 (예: 2026): "))
    month = int(input("예측할 월을 입력하세요 (1~12): "))
    return year, month

In [ ]:
# 모델1 one-hot encodiong
# 모델1의 입력값을 변환함
# MONTH 컬럼을 One-Hot Encoding 후, model1_columns에 맞게 순서 및 누락된 열 보완
def convert_to_model1_input(year, month, model1_columns):
    df = pd.DataFrame([[year, month]], columns=['YEAR', 'MONTH'])

    df['YEAR_SINCE_2000'] = year - 2000
    df['LINEAR_TEMP_TREND'] = 0.017 * df['YEAR_SINCE_2000']
    df['YEAR_MONTH_INDEX'] = df['YEAR_SINCE_2000'] * 12 + df['MONTH']
    df['YEAR_SQUARED'] = df['YEAR_SINCE_2000'] ** 2
    # One-hot encoding
    df = pd.get_dummies(df, columns=['MONTH'], prefix='MONTH')

    # 누락된 컬럼을 모두 0.0으로 채우기
    for col in model1_columns:
        if col not in df.columns:
            df[col] = 0.0

    # 컬럼 순서 맞추기
    df = df[model1_columns]
    return df

In [ ]:
def predict_climate(model1, X_input1):
    y_pred = model1.predict(X_input1)[0]

    max_temp = y_pred[0]
    min_temp = y_pred[1]
    wind_speed = y_pred[2]
    temp_range = y_pred[3]
    precipitation = y_pred[4]
    return [max_temp, min_temp, wind_speed, temp_range, precipitation]

In [ ]:
# 모델1 아웃풋+추가정보를 모델2 입력값으로 변환
def convert_to_model2_input(model1_output, month, model2_columns):
    max_temp, min_temp, wind_speed, temp_range, precipitation = model1_output

    season = convert_month_to_season(month)
    season_cols = ['SEASON_Spring', 'SEASON_Summer', 'SEASON_Fall', 'SEASON_Winter']
    season_encoding = {col: 1 if col.endswith(season) else 0 for col in season_cols}

    input_dict = {
        'MAX_TEMP': max_temp,
        'MIN_TEMP': min_temp,
        'AVG_WIND_SPEED': wind_speed,
        'TEMP_RANGE': temp_range,
        'PRECIPITATION': precipitation,
        **season_encoding
    }
    input_df = pd.DataFrame([input_dict])
    input_df = input_df.reindex(columns=model2_columns, fill_value=0)
    return input_df

In [ ]:
def convert_month_to_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

In [ ]:
# 모델2로 산불 확률 예측
def predict_fire_probability(model2, X_input2):
    prob = model2.predict_proba(X_input2)[0][1]
    return prob

In [ ]:
def run_fire_risk_prediction(model1, model2, model1_columns, model2_columns):
    year, month = get_user_input()

    X_model1 = convert_to_model1_input(year, month, model1_columns)
    climate_pred = predict_climate(model1, X_model1)

    X_model2 = convert_to_model2_input(climate_pred, month, model2_columns)

    fire_prob = predict_fire_probability(model2, X_model2)
    print(f"\n✅ 예측 결과: {year}년 {month}월의 산불 발생 확률은 **{fire_prob*100:.2f}%** 입니다.")


In [ ]:
model1 = joblib.load("final_stacking_catboost_model.pkl")
model2 = joblib.load("final_model_2.pkl")
# 모델1: YEAR, MONTH를 입력받아 기상 요소 예측
model1_columns = ['YEAR_SINCE_2000'] + [f'MONTH_{i}' for i in range(1, 13)] + ['LINEAR_TEMP_TREND', 'YEAR_MONTH_INDEX', 'YEAR_SQUARED']
model2_columns = [
    'PRECIPITATION', 'MAX_TEMP', 'MIN_TEMP', 'AVG_WIND_SPEED', 'TEMP_RANGE',
    'SEASON_Fall', 'SEASON_Spring', 'SEASON_Summer', 'SEASON_Winter'
]

In [ ]:
run_fire_risk_prediction(model1, model2, model1_columns, model2_columns)

예측할 연도를 입력하세요 (예: 2026): 2049
예측할 월을 입력하세요 (1~12): 4

✅ 예측 결과: 2049년 4월의 산불 발생 확률은 **14.22%** 입니다.


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(
